In [109]:
import pandas as pd
from xgboost import XGBClassifier
import data_processing as dp

In [110]:
d = dp.DataProcessing()
kenpom = d.process_kenpom()
d.process_tourney()
model_data = d.get_data()

c:\Users\jsull\Python Projects\MarchMadness\data_processing.py:217: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  matchup_stats[f'{stat_name}_diff'] = (
c:\Users\jsull\Python Projects\MarchMadness\data_processing.py:217: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  matchup_stats[f'{stat_name}_diff'] = (
c:\Users\jsull\Python Projects\MarchMadness\data_processing.py:217: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join

In [111]:
features = ['higher_seed_num', 'higher_record', 'lower_seed_num', 'lower_record', 'Tempo_diff', 'AdjTempo_diff', 'OE_diff',
       'AdjOE_diff', 'DE_diff', 'AdjDE_diff', 'AdjEM_diff', 'OffFT_diff',
       'Off2PtFG_diff', 'Off3PtFG_diff', 'DefFT_diff', 'Def2PtFG_diff',
       'Def3PtFG_diff', 'eFGPct_diff', 'TOPct_diff', 'ORPct_diff',
       'FTRate_diff', 'FG2Pct_diff', 'FG3Pct_diff', 'FTPct_diff',
       'BlockPct_diff', 'OppFG2Pct_diff', 'OppFG3Pct_diff', 'OppFTPct_diff',
       'OppBlockPct_diff', 'FG3Rate_diff', 'OppFG3Rate_diff', 'ARate_diff',
       'OppARate_diff', 'StlRate_diff', 'OppStlRate_diff',
       'Adjusted Temo_diff', 'Raw Tempo_diff',
       'Adjusted Offensive Efficiency_diff', 'Raw Offensive Efficiency_diff',
       'Adjusted Defensive Efficiency_diff', 'Raw Defensive Efficiency_diff']

In [112]:
mm_25 = pd.read_csv('competition_submission.csv').drop(columns=['Unnamed: 0'])
season_2025 = kenpom[kenpom['Season'] == 2025]
games = (mm_25.merge(season_2025, left_on='higher_seed', right_on='TeamName', how='left')
 .merge(season_2025, left_on='lower_seed', right_on='TeamName', how='left', suffixes=('_higher', '_lower'))).drop(columns=['TeamName_higher', 'TeamName_lower', 'Season_higher', 'Season_lower'])

h_columns = [col for col in games.columns if col.endswith('_higher')]
l_columns = [col for col in games.columns if col.endswith('_lower')]

games[h_columns + l_columns] = games[h_columns + l_columns].apply(pd.to_numeric, errors='coerce')

for col_h in h_columns:
    col_l = col_h.replace('_higher', '_lower')
    if col_l in l_columns:
        # Compute differences row-wise
        stat_name = col_h.replace('_higher', '')  # Remove _W to get the base stat name
        games[f'{stat_name}_diff'] = (
            games[col_h] - games[col_l]
        )

tournament = games[features]

C:\Users\jsull\AppData\Local\Temp\ipykernel_2136\1949778999.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  games[f'{stat_name}_diff'] = (
C:\Users\jsull\AppData\Local\Temp\ipykernel_2136\1949778999.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  games[f'{stat_name}_diff'] = (
C:\Users\jsull\AppData\Local\Temp\ipykernel_2136\1949778999.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at 

In [113]:
X = model_data[features]
y = model_data['HigherSeedWin']

In [114]:
model = XGBClassifier(booster = 'gbtree', objective = 'binary:logistic', device = 'gpu', nthread = -1, subsample = 1, 
                      min_child_weight = 4, max_depth = 7, max_delta_step = 3, reg_lambda = 5, gamma = 4, eta = 1, alpha = 4)

model.fit(X, y)

XGBClassifier(alpha=4, base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='gpu', early_stopping_rounds=None,
              enable_categorical=False, eta=1, eval_metric=None,
              feature_types=None, gamma=4, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=3, max_depth=7,
              max_leaves=None, min_child_weight=4, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, ...)

In [115]:
predictions = model.predict_proba(tournament)
mm_25['Pred'] = predictions[:,1]
mm_25

,higher_seed,higher_seed_num,higher_record,lower_seed,lower_seed_num,lower_record,Pred
0,Duke,1,0.911765,Florida,1,0.882353,0.314868
1,Duke,1,0.911765,Houston,1,0.882353,0.235282
2,Duke,1,0.911765,Auburn,1,0.848485,0.420117
3,Duke,1,0.911765,St. John's,2,0.882353,0.932571
4,Duke,1,0.911765,Michigan St.,2,0.818182,0.948317
...,...,...,...,...,...,...,...
2273,American,16,0.647059,Alabama St.,16,0.558824,0.188110
2274,American,16,0.647059,Saint Francis,16,0.484848,0.701547
2275,Mount St. Mary's,16,0.647059,Alabama St.,16,0.558824,0.515122
2276,Mount St. Mary's,16,0.647059,Saint Francis,16,0.484848,0.816839


In [116]:
def lookup_game(higher_seed, lower_seed):
    return mm_25[(mm_25['higher_seed'] == higher_seed) & (mm_25['lower_seed'] == lower_seed)]

In [127]:
lookup_game("Duke", "Florida")

,higher_seed,higher_seed_num,higher_record,lower_seed,lower_seed_num,lower_record,Pred
0,Duke,1,0.911765,Florida,1,0.882353,0.314868
